In [1]:
import numpy as np
import torch
from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score, classification_report, f1_score
from transformers import ViTForImageClassification, ViTImageProcessor
from datasets import load_dataset
from torch.utils.data import DataLoader

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
dataset = load_dataset("uoft-cs/cifar10", split="test")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/120M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/23.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

## ViT

In [17]:
model_id = "nateraw/vit-base-patch16-224-cifar10"
model = ViTForImageClassification.from_pretrained(model_id)
model.to(device)
model.eval()

processor = ViTImageProcessor.from_pretrained(model_id)

config.json:   0%|          | 0.00/918 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/343M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

In [ ]:
def preprocess_function(examples):
    images = [img.convert("RGB") for img in examples["img"]]
    inputs = processor(images, return_tensors="pt")
    inputs["labels"] = examples["label"]
    return inputs

tokenized_dataset = dataset.map(
    preprocess_function,
    batched=True,
    batch_size=32,
    remove_columns=dataset.column_names,
)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [19]:
tokenized_dataset.set_format("torch")
test_dataloader = DataLoader(tokenized_dataset, batch_size=64, shuffle=False)

In [20]:
all_preds = []
all_labels = []

for batch in tqdm(test_dataloader):
    pixel_values = batch['pixel_values'].to(device)
    labels = batch['labels']

    with torch.no_grad():
        outputs = model(pixel_values)
        logits = outputs.logits

    predictions = torch.argmax(logits, dim=-1).cpu().numpy()

    all_preds.extend(predictions)
    all_labels.extend(labels.numpy())

  0%|          | 0/157 [00:00<?, ?it/s]

In [21]:
y_true = np.array(all_labels)
y_pred = np.array(all_preds)

accuracy = accuracy_score(y_true, y_pred)
print(f'accuracy: {accuracy}')

f1 = f1_score(y_true, y_pred, average='weighted')
print(f"weighted f1: {f1}")

target_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
report = classification_report(y_true, y_pred, target_names=target_names, digits=4)
print(report)

accuracy: 0.9852
weighted f1: 0.9851943763781306
              precision    recall  f1-score   support

    airplane     0.9939    0.9830    0.9884      1000
  automobile     0.9783    0.9910    0.9846      1000
        bird     0.9910    0.9940    0.9925      1000
         cat     0.9735    0.9550    0.9642      1000
        deer     0.9920    0.9870    0.9895      1000
         dog     0.9607    0.9790    0.9698      1000
        frog     0.9901    0.9980    0.9940      1000
       horse     0.9950    0.9910    0.9930      1000
        ship     0.9930    0.9930    0.9930      1000
       truck     0.9849    0.9810    0.9830      1000

    accuracy                         0.9852     10000
   macro avg     0.9852    0.9852    0.9852     10000
weighted avg     0.9852    0.9852    0.9852     10000



## MobileNetv2

In [4]:
from transformers import AutoModelForImageClassification, AutoImageProcessor

In [10]:
model_id = "AiresPucrs/Mobilenet-v2-CIFAR-10"
model = AutoModelForImageClassification.from_pretrained(model_id)
model.to(device)
model.eval()

processor = AutoImageProcessor.from_pretrained(model_id)

preprocessor_config.json:   0%|          | 0.00/434 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [ ]:
def preprocess_function(examples):
    images = [img.convert("RGB") for img in examples["img"]]
    inputs = processor(images, return_tensors="pt")
    inputs["labels"] = examples["label"]
    return inputs

tokenized_dataset = dataset.map(
    preprocess_function,
    batched=True,
    batch_size=32,
    remove_columns=dataset.column_names,
)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset.set_format("torch")
test_dataloader = DataLoader(tokenized_dataset, batch_size=64, shuffle=False)

In [15]:
all_preds = []
all_labels = []

for batch in tqdm(test_dataloader):
    pixel_values = batch['pixel_values'].to(device)
    labels = batch['labels']

    with torch.no_grad():
        outputs = model(pixel_values)
        logits = outputs.logits

    predictions = torch.argmax(logits, dim=-1).cpu().numpy()

    all_preds.extend(predictions)
    all_labels.extend(labels.numpy())

  0%|          | 0/157 [00:00<?, ?it/s]

In [16]:
y_true = np.array(all_labels)
y_pred = np.array(all_preds)

accuracy = accuracy_score(y_true, y_pred)
print(f'accuracy: {accuracy}')

f1 = f1_score(y_true, y_pred, average='weighted')
print(f"weighted f1: {f1}")

target_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
report = classification_report(y_true, y_pred, target_names=target_names, digits=4)
print(report)

accuracy: 0.9187
weighted f1: 0.9187221228260388
              precision    recall  f1-score   support

    airplane     0.9456    0.9040    0.9243      1000
  automobile     0.9268    0.9750    0.9503      1000
        bird     0.8557    0.9370    0.8945      1000
         cat     0.8486    0.8520    0.8503      1000
        deer     0.9442    0.8800    0.9110      1000
         dog     0.8871    0.8800    0.8835      1000
        frog     0.9361    0.9530    0.9445      1000
       horse     0.9600    0.9360    0.9478      1000
        ship     0.9172    0.9750    0.9452      1000
       truck     0.9803    0.8950    0.9357      1000

    accuracy                         0.9187     10000
   macro avg     0.9202    0.9187    0.9187     10000
weighted avg     0.9202    0.9187    0.9187     10000

